# Percetptrone

## Laboratorio di Python

### Esperimento 1: Implementazione di un perceptrone

In questo esperimento sviluppiamo una implementazione del perceptrone senza fare uso di librerie esterne e lo useremo in un problema di classificazione con classi linearmente separabili.
Le sole librerie usata sono **random** per la generazione di numeri casuali per simulare i dati e **matplotlib** per graficare i risultati.
I dati simulati sono costruti per essere linearmente separabili.

In [ ]:
import random
import matplotlib.pyplot as plt

# -----------------------
# 1. Generazione del dataset
# -----------------------

random.seed(42)
n_samples = 100
X = []
y = []

for _ in range(n_samples):
    x1 = random.uniform(0, 10)
    x2 = random.uniform(0, 10)
    label = 1 if x1 + x2 > 10 else 0  # Separazione lineare lungo la diagonale x1 + x2 = 10
    X.append([x1, x2])
    y.append(label)

# Suddividiamo manualmente in training e test set
split_index = int(0.8 * n_samples)
X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# -----------------------
# 2. Funzioni del Percettrone
# -----------------------

def step_function(z):
    return 1 if z >= 0 else 0

def predict(x, weights, bias):
    z = sum(w * xi for w, xi in zip(weights, x)) + bias
    return step_function(z)

def train_perceptrone(X_train, y_train, learning_rate=0.1, epochs=10):
    n_features = len(X_train[0])
    weights = [0.0 for _ in range(n_features)]
    bias = 0.0

    for epoch in range(epochs):
        for x, label in zip(X_train, y_train):
            y_pred = predict(x, weights, bias)
            error = label - y_pred
            # Aggiornamento pesi e bias
            for i in range(n_features):
                weights[i] += learning_rate * error * x[i]
            bias += learning_rate * error
    return weights, bias

# -----------------------
# 3. Addestramento del modello
# -----------------------

weights, bias = train_perceptrone(X_train, y_train, learning_rate=0.1, epochs=30)

# -----------------------
# 4. Valutazione e Matrice di Confusione
# -----------------------

TP = FP = TN = FN = 0
y_pred_test = []

for x, label in zip(X_test, y_test):
    y_hat = predict(x, weights, bias)
    y_pred_test.append(y_hat)
    if y_hat == 1 and label == 1:
        TP += 1
    elif y_hat == 1 and label == 0:
        FP += 1
    elif y_hat == 0 and label == 0:
        TN += 1
    elif y_hat == 0 and label == 1:
        FN += 1

print("Matrice di Confusione (senza librerie esterne):")
print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
accuracy = (TP + TN) / len(y_test)
print(f"Accuratezza: {accuracy:.2f}")

# -----------------------
# 5. Visualizzazione
# -----------------------

x1_pos = [X_test[i][0] for i in range(len(y_test)) if y_test[i] == 1]
x2_pos = [X_test[i][1] for i in range(len(y_test)) if y_test[i] == 1]
x1_neg = [X_test[i][0] for i in range(len(y_test)) if y_test[i] == 0]
x2_neg = [X_test[i][1] for i in range(len(y_test)) if y_test[i] == 0]

plt.figure(figsize=(8, 6))
plt.scatter(x1_pos, x2_pos, c='green', label='Classe 1 (Reale)', marker='x')
plt.scatter(x1_neg, x2_neg, c='blue', label='Classe 0 (Reale)', marker='o')

# Linea di decisione: w1*x1 + w2*x2 + b = 0 => x2 = -(w1*x1 + b)/w2
x1_vals = [i for i in range(0, 11)]
if weights[1] != 0:
    x2_vals = [-(weights[0]*x + bias)/weights[1] for x in x1_vals]
    plt.plot(x1_vals, x2_vals, '--', color='black', label='Confine decisionale')

plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Percettrone - Separazione tra classi")
plt.legend()
plt.grid(True)
plt.show()

Il codice Python usato per l'implementazione del perceptrone e per il grafico dei risultati segue la seguente logica:

1. **Generazione dati**
Simuliamo 100 punti 2D. I punti per cui $x_1 + x_2 > 10$ sono etichettati come **classe 1**, altrimenti **classe 0**.

2. **Implementazione del percettrone**
- **`step_function(z)`**: restituisce 1 se la somma è maggiore o uguale a zero, altrimenti 0.
- **`predict(x, weights, bias)`**: calcola il valore di $z$ e applica la funzione di attivazione.
- **`train_perceptrone()`**: implementa l'algoritmo di apprendimento con aggiornamento dei pesi.

3. **Addestramento**
Il modello viene addestrato per 20 epoche (ripetizioni del dataset) su 80% dei dati.

4. **Matrice di confusione**
Calcolo dei 4 elementi della matrice di confusione:
- **TP**: (True Positive = Vero Positivo) predetto 1 e vero 1
- **TN**: (True Negative = Vero Negativo) predetto 0 e vero 0
- **FP**: (False Positive = Falso Positivo) predetto 1 e vero 0 (falso allarme)
- **FN**: (False Negative = Falso Negativo) predetto 0 e vero 1 (falso negativo)

5. **Visualizzazione**
Il grafico mostra:
- I punti veri (blu per classe 0, verde per classe 1).
- Il **confine decisionale** calcolato con la formula della retta.

**Risultato atteso**

Con dati così semplici e ben separabili, il percettrone dovrebbe ottenere una **accuratezza alta (quasi 100%)**.

### Esperimento 2: il perceptrone bel caso di classi concentriche {#sec-lab-perceptrone-classi-concentriche}

Il codice Python che segue implementa un semplice modello di classificatore a **perceptrone** usando la sola libreria random per la generazione di numeri casuali per simulare i dati e matplotlib per graficare i risultati.
I dati simulati sono distribuiti su due corone circolari concentriche e quindi non linearmente separabili.

In [ ]:
import math
import random
import matplotlib.pyplot as plt

# -------------------------------
# 1. Generazione del dataset circolare
# -------------------------------

def genera_cerchi(n_samples):
    X = []
    y = []
    for _ in range(n_samples):
        r = random.uniform(0, 1)
        angle = random.uniform(0, 2 * math.pi)
        if r < 0.5:
            radius = random.uniform(1, 2)
            label = 0
        else:
            radius = random.uniform(3, 4)
            label = 1
        x1 = radius * math.cos(angle)
        x2 = radius * math.sin(angle)
        X.append([x1, x2])
        y.append(label)
    return X, y

random.seed(42)
X, y = genera_cerchi(200)

# Suddividiamo in training e test set
split_index = int(0.8 * len(X))
X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# -------------------------------
# 2. Percettrone da zero
# -------------------------------

def step_function(z):
    return 1 if z >= 0 else 0

def predict(x, weights, bias):
    z = sum(w * xi for w, xi in zip(weights, x)) + bias
    return step_function(z)

def train_perceptrone(X, y, learning_rate=0.1, epochs=20):
    n_features = len(X[0])
    weights = [0.0 for _ in range(n_features)]
    bias = 0.0
    for epoch in range(epochs):
        for xi, yi in zip(X, y):
            y_pred = predict(xi, weights, bias)
            error = yi - y_pred
            for i in range(n_features):
                weights[i] += learning_rate * error * xi[i]
            bias += learning_rate * error
    return weights, bias

# Addestramento
weights, bias = train_perceptrone(X_train, y_train)

# -------------------------------
# 3. Valutazione
# -------------------------------

TP = FP = TN = FN = 0
y_pred_test = []

for x, label in zip(X_test, y_test):
    y_hat = predict(x, weights, bias)
    y_pred_test.append(y_hat)
    if y_hat == 1 and label == 1:
        TP += 1
    elif y_hat == 1 and label == 0:
        FP += 1
    elif y_hat == 0 and label == 0:
        TN += 1
    elif y_hat == 0 and label == 1:
        FN += 1

print("Matrice di confusione:")
print(f"TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}")
accuracy = (TP + TN) / len(y_test)
print(f"Accuratezza: {accuracy:.2f}")

# -------------------------------
# 4. Visualizzazione
# -------------------------------

# Colori reali
x1_pos = [X_test[i][0] for i in range(len(y_test)) if y_test[i] == 1]
x2_pos = [X_test[i][1] for i in range(len(y_test)) if y_test[i] == 1]
x1_neg = [X_test[i][0] for i in range(len(y_test)) if y_test[i] == 0]
x2_neg = [X_test[i][1] for i in range(len(y_test)) if y_test[i] == 0]

plt.figure(figsize=(8, 6))
plt.scatter(x1_pos, x2_pos, color='green', label='Classe 1 (Reale)', marker='x')
plt.scatter(x1_neg, x2_neg, color='blue', label='Classe 0 (Reale)', marker='o')

# Linea di decisione (valida solo per separazione lineare)
x_vals = [i / 10.0 for i in range(-40, 41)]
if weights[1] != 0:
    y_vals = [-(weights[0]*x + bias)/weights[1] for x in x_vals]
    plt.plot(x_vals, y_vals, '--', color='black', label='Confine decisionale')

plt.title("Percettrone su dati non linearmente separabili")
plt.xlabel("x1")
plt.ylabel("x2")
plt.legend()
plt.grid(True)
plt.show()

L'esito dell'esperimento mostra chiaramente ciò che già sapevamo bene:
Il **percettrone può apprendere solo confini lineari**. 
In questo caso:

- I dati sono disposti in **cerchi concentrici**, cioè non possono essere separati con una retta.
- Il percettrone cerca una retta per dividere i dati… ma non riesce e commette molti errori.
- L’**accuratezza è bassa** (minore del 50%).
- Il grafico mostra che la **linea di decisione** non riesce a separare correttamente le due classi.

In casi come questo servono modelli di reti neurali più avanzati come le **reti neurali multistrato (MLP)** .